# Adversarial Robustness Audit

This notebook demonstrates the adversarial robustness audit pipeline using the `audit_tool.robustness` module.  
All heavy lifting (attacks, evaluation, plotting, report generation) is implemented in that module.  
Below we load data and models, configure attack parameters, run the audit, and generate an HTML report.

In [1]:
# Credit Risk Audit Tool - Adversarial Robustness Phase
# Dataset: Prosper Loan Data

import sys
import os
# Add the project root to the Python path so we can import audit_tool
sys.path.append(os.path.abspath('..'))

import pandas as pd
import numpy as np
import joblib

# Import our robustness functions
from audit_tool import robustness as rb

/home/celia/Desktop/tfm-credit-risk-audit/venv/lib/python3.10/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


## 1. Load Data and Models

Define paths for the pre-processed test dataset and the serialized model files.


In [ ]:
X_path = '../data/processed/prosperloan/X_test_lasso.csv'
y_path = '../data/processed/prosperloan/y_test_lasso.csv'
model_paths = {
    'logreg': '../models/prosperloan/logisticregression.pkl',
    'rf':     '../models/prosperloan/randomforest.pkl',
    'xgb':    '../models/prosperloan/xgboost.pkl'
}

X_test, y_test, feature_names = rb.load_data(X_path, y_path)
models = rb.load_models(model_paths)
art_classifiers = rb.wrap_art_classifiers(models)

print(f"Loaded X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"Models available for audit: {list(models.keys())}")


## 2. Configure Attack Parameters and Report Settings

- Choose a list of FGSM epsilon values to test  
- Specify output directories and report format


In [ ]:
epsilons    = [0.0, 0.1, 0.2]             # FGSM attack strengths
graphics_dir = 'figures'                  # directory to save plots
report_file  = 'robustness_report.html'   # output report filename
report_fmt   = 'html'                     # choose 'html' or 'md'

# Ensure output directory exists
os.makedirs(graphics_dir, exist_ok=True)

## 3. Run Full Audit and Generate Report

This will:
- Generate adversarial examples at each epsilon  
- Evaluate each model (accuracy & F1) on adversarial inputs  
- Analyze top-perturbed features  
- Plot and save graphics  
- Compile a complete HTML report with embedded images and tables


In [ ]:
rb.generate_report(
    X_orig=X_test,
    y_true=y_test,
    art_clfs=art_classifiers,
    epsilons=epsilons,
    feature_names=feature_names,
    figures_dir=graphics_dir,
    report_path=report_file,
    fmt=report_fmt
)


## 4. View Report

After execution, open the generated report (`robustness_report.html`) in your browser to explore:
- Executive summary  
- Detailed performance plots per epsilon  
- Top feature perturbation charts and tables  
- Recommendations for improving model robustness
